<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       DIFF and UNDIFF function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>DIFF</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The DIFF() function transforms a stationary, seasonal, or non-stationary time series into a differenced time series by performing both status-quo time series differencing, seasonal based differencing, and multiplicative transforms. Thus, the output of this transform function is always a new time series.</p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following procedure is an example of how to use DIFF() function::</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Detect the unit roots using DickeyFuller() function.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use DIFF() function to eliminate unit roots.</li>
</p>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>UNDIFF</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The UNDIFF() function is the reverse of the DIFF() function. It takes in a previously differenced series processed by DIFF(), and produces the original series that existed prior to the differencing.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    DIFF,
    UNDIFF,
    Figure,
    plot,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_DIFF_UNDIFF_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_SalesForecastingUAF_cloud');"        # Takes 1 minute
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_SalesForecastingUAF_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
sales_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Weekly_Sales'))
feature_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Features')).drop(['IsHoliday'], axis=1)
store_data = DataFrame(in_schema('DEMO_SalesForecastingUAF', 'Stores'))

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will join datasets to create the Analytic Data Set using tdml for basic dataframe manipulations</p>

In [ ]:
# Join store_data with sales_data
sales_data = (
    sales_data.join(store_data, on='Store', how='left', lprefix='t1', rprefix='t2')
    .drop(['t2_Store'], axis=1)
)
sales_data = sales_data.assign(Store=sales_data['t1_Store'])
sales_data = sales_data.drop(['t1_Store'], axis=1)

# Join feature_data with sales_data
sales_data = (
    sales_data.join(feature_data, on=['Store', 'Date'], how='left', lprefix='t1', rprefix='t3')
    .drop(['t3_Store'], axis=1)
    .drop(['t3_Date'], axis=1)
)
sales_data = sales_data.assign(Store=sales_data['t1_Store'])
sales_data = sales_data.assign(Date=sales_data['t1_Date'])
sales_data = sales_data.drop(['t1_Store'], axis=1)
sales_data = sales_data.drop(['t1_Date'], axis=1)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will creating time series identifier for partitioning the data. We will join the department and store ID which provides a unique ID for every time series to create a column for partitioning.</p>

In [ ]:
# Assign a new column 'idcols' based on string concatenation
sales_data = sales_data.assign(idcols=sales_data.Dept.str.strip() + '-' + sales_data.Store.str.strip())
sales_data = sales_data.assign(idcols=sales_data.idcols.cast(type_=VARCHAR(10)))

# Check the shape of the DataFrame
sales_data.shape

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. DIFF</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute DIFF() function with TDSeries having REAL payload content to transform time series into a differenced time series.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the convolve function.</p>


In [ ]:
sales_data

In [ ]:
data_series_df_real = TDSeries(data=sales_data,
                              id="idcols",
                              row_index="times",
                              row_index_style="TIMECODE",
                              payload_field="Weekly_Sales",
                              payload_content="REAL") 

In [ ]:
uaf_out_1 = DIFF(data=data_series_df_real,
                     lag=1,
                     differences=2,
                     seasonal_multiplier=0)
diff_df = uaf_out_1.result
diff_df

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. UNDIFF</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The UNDIFF() function will retrieve the original series that existed prior to the differencing by taking the differenced series processed by DIFF() as input.</p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first step is to create teradataml TDSeries over the output generated from DIFF.</p>

In [ ]:
# Create teradataml TDSeries objects.
td_series1 = TDSeries(data=uaf_out_1.result,
                          id="idcols",
                          row_index="ROW_I",
                          row_index_style="SEQUENCE",
                          payload_field="OUT_Weekly_Sales",
                          payload_content="REAL"
                          )

In [ ]:
uaf_out = UNDIFF(data1=td_series1,
                     data2=data_series_df_real,
                     lag=1,
                     differences=1,
                     seasonal_multiplier=0,
                     input_fmt_input_mode="MATCH")
 
# Print the result DataFrame.
undiff_df=uaf_out.result
undiff_df

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_SalesForecastingUAF');" 
#Takes 45 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>DIFF function reference: <a href = 'https://docs.teradata.com/search/all?query=DIFF&content-lang=en-US'>here</a></li>
    <li>UNDIFF function reference: <a href = 'https://docs.teradata.com/search/all?query=UNDIFF&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>